In [1]:
print(' ~ ~ ~ ~ ~ ~ ~ ~ OM SHRI GANESHAAYA NAMAHA ~ ~ ~ ~ ~ ~ ~ ~ ~ ')

 ~ ~ ~ ~ ~ ~ ~ ~ OM SHRI GANESHAAYA NAMAHA ~ ~ ~ ~ ~ ~ ~ ~ ~ 


##### Task 1: Extract reviews of a given product

#### Import required libraries

In [2]:
import pandas as pd
import numpy as np
import traceback
import re
import requests

from tqdm.notebook import tqdm

from bs4 import BeautifulSoup 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options  
from selenium.common.exceptions import TimeoutException

import time

# from clear_cache import clear_firefox_cache

## <font color='a33668'> <b> Web-scraping for product reviews (Amazon) </b> </font>

In [3]:
headers = {'authority': 'www.amazon.in',
           'accept-language': 'en-US,en;q=0.9',
           'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

In [4]:
def get_soup(url):
    # Path to the geckodriver executable
    geckodriver_path = '/ssg/bin/geckodriver.exe'
    # Driver options
    options = Options()   
    options.add_argument('--headless')
    options.add_argument('--start-maximized')
    
    # Create a Firefox driver
    driver = webdriver.Firefox(options=options)
    
    driver.get(url)
    
    # Wait for the entire page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    return soup

reviewlist = []

def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})

    try:
        for item in reviews:
            review = {'Rating':float(item.find('i', {'data-hook': 'cmps-review-star-rating'}).text.replace('out of 5 stars', '').strip()),
                      'Title': item.find('span', {'data-hook': 'review-title'}).text.strip(),
                      'Review':item.find('span', {'data-hook': 'review-body'}).text.strip()}
            reviewlist.append(review)
    except Exception as e:
        traceback.print_exc()
        pass

In [5]:
for x in tqdm(range(1)):
    url_str = f'https://www.amazon.in/product-reviews/B071F49Q2P/ref=cm_cr_unknown?ie=UTF8r&reviewerType=all_reviews&pageNumber={x+1}'
    soup = get_soup(url_str) 
    get_reviews(soup)
    if not soup.find('li', {'class': 'a-disabled a-last'}):
        pass
    else:
        break

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
reviewlist

[{'Rating': 5.0,
  'Title': 'Good products refurbished  to almost new condition',
  'Review': 'all 3 laptops worked as expected  good batteries, nice screen and fast SSD would buy again'},
 {'Rating': 5.0,
  'Title': 'Very Satisfied',
  'Review': 'It seems most "computer" refurbs are offered here on Amazon through a variety of different sellers under the same listing?  So, for the record, my order was fulfilled through "Budget Business Computers".  I\'ve had the unit for almost a month now.  Unit arrived with a few very minor "aesthetic" issues - a couple of small dings and a small scratch on the cover - but all technical details matched the product description, fresh install of Windows 10 Pro, and the computer itself works flawlessly to date.  FWIW, runs a bit "warm", as thinner units tend to, and I\'d say battery life is very  good, but not great.  Overall, I\'m very satisfied with my purchase.  Great price, quick delivery (International delivery, no less) and in general a decent lit

## <font color='a33668'> <b> Web-scraping for product reviews (Dell) </b> </font>

### <font color='a33668'> Scraping code </font>

In [7]:
# Function to scroll to the host element of the Shadow DOM
def check_shadow_dom(driver, host_element, max_retries=5):
    retry_count = 0
    css_selector = "#cfBazaarReviewWrapper > div:nth-child(2)"
    while retry_count < max_retries:
        print(f'Checking Shadow DOM ...')
        print(f'Trying {retry_count+1} of {max_retries} ...')
        try:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", host_element)
            has_shadow_dom = driver.execute_script("return arguments[0].shadowRoot !== null && arguments[0].shadowRoot !== 'None'", host_element)
            if has_shadow_dom: 
                return has_shadow_dom
            else: 
                time.sleep(1) # Adjust the sleep duration as required.
        except TimeoutException:
            print("Timed out waiting for element.")
        retry_count += 1
    print("Reached maximum retries. Unable to determine if element has shadow DOM.")
    return False

def scrape_page(n):
    s='' # string for populating the extracted data
    try:
        # url = "https://www.dell.com/en-in/work/shop/business-laptop-notebook-computers/latitude-7440-laptop/spd/latitude-14-7440-2-in-1-laptop/n001l7440017in9_vp#ratings_section"
        url1 = "https://www.dell.com/en-in/work/shop/business-laptop-notebook-computers/latitude-7440-laptop/spd/latitude-14-7440-2-in-1-laptop/n001l7440017in9_vp?bvstate=pg:"
        url3 = "/ct:r#ratings_section"
        url = f'{url1}{n}{url3}'
    
        options = Options()
        # options.add_argument('--headless')
        # options.add_argument('--start-maximized')
    
        # Create a Firefox driver
        driver = webdriver.Firefox(options=options)

        print(f'Getting ... {url}')
        driver.get(url)
    
        # Scroll to the element that is the host of the Shadow DOM
        css_selector = "#cfBazaarReviewWrapper > div:nth-child(2)"
        wait = WebDriverWait(driver, 10)
        host_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", host_element)
    
        # print(f'host_element: {host_element}')
        
        # Use JavaScript to check if the element has a shadow root
        has_shadow_dom = check_shadow_dom(driver, host_element)
        
        if has_shadow_dom :
            print('Shadow DOM found.')
            max_retries = 5
            retry_count = 0
            inner_element = None
            while retry_count < max_retries:
                    # JavaScript to scroll to and get the handle of the inner element ('#reviews_container') within the Shadow DOM
                    script = """
                    const hostElement = arguments[0];
                    const innerElementSelector = '#reviews_container';
                    const innerElement = hostElement.shadowRoot.querySelector(innerElementSelector);
                    if (innerElement) {
                        innerElement.scrollIntoView({behavior: 'smooth', block: 'center'});
                        return innerElement;
                    }
                    return null;
                    """
                    # Execute the JavaScript to scroll to the inner element within the Shadow DOM and get its handle
                    inner_element = driver.execute_script(script, host_element)
        
                    if inner_element is not None:
                        # Inner element found, break out of the loop
                        break
                    else:
                        # Increment retry count and wait before retrying
                        retry_count += 1
                        time.sleep(1)  # Adjust the sleep duration as needed
                        
        # If Shadow DOM does not exist then raise an exception
        else:
            raise Exception('Host element (\"#cfBazaarReviewWrapper > div:nth-child(2)\") does not have a Shadow DOM, or it is closed.')
    
        print("Shadow DOM inner element ('#reviews_container') found.")
        # print("inner_element is: ", inner_element)
        if inner_element:
            
            # Find all 'section' elements with ids starting with 'bv-review-'
            review_elements = inner_element.find_elements(By.CSS_SELECTOR, "section[id^='bv-review-']")
    
            # print(review_elements) 
            
            # Loop through the collection of 'section' elements
            for element in review_elements:
                # Scroll the section element into view
                driver.execute_script("arguments[0].scrollIntoView(true);", element)
        
                # Find the 'abbr' element within each 'section' element
                abbr_element = element.find_element(By.TAG_NAME, 'abbr')
                
                # Get the title attribute of the 'abbr' element
                abbr_title = abbr_element.get_attribute("title")
                # print("***** abbr_title: ", abbr_title)
                s += f'\n {abbr_title[0]}|'
    
                # Find the div element with attribute data-bv-v='contentItem' under the 'section' element
                content_item_div = element.find_element(By.CSS_SELECTOR, "div[data-bv-v='contentItem']")
    
                # Find the div element with attribute data-bv-v='contentHeader' under the 'contentItem' div
                content_header_div = content_item_div.find_element(By.CSS_SELECTOR, "div[data-bv-v='contentHeader']")
                
                # Find the h3 element under the 'contentHeader' div
                h3_element = content_header_div.find_element(By.TAG_NAME, "h3")
    
                # print("****** h3_element.text: ", h3_element.text)
                s += f'{h3_element.text}|'
    
                # Find the div tag with attribute data-bv-v='contentSummary' within the content item div
                content_summary_div = content_item_div.find_element(By.CSS_SELECTOR, "div[class='bv-rnr__sc-16dr7i1-3 kPwaVD']")
    
                # print("***** content_summary_div.text: ", content_summary_div.text)
                s += f'{content_summary_div.text}'
        else:
            print("Failed to find \'reviews_container\' element.")
    
    except Exception as e:
        print ('General Error: ', e)
        return 'Retrieval error.'
    
    finally:
        try:
            # Clear the cache using JavaScript execution
            driver.execute_script("window.localStorage.clear();")
            driver.execute_script("window.sessionStorage.clear();")
            driver.execute_script("window.location.reload();")
        except Exception as e:
            print("Error clearing cache:", e)
        driver.quit()
    print(f'Relevant information from page {n} retrieved.')
    return s

### <font color='a33668'> Calling the function with page number as param </font>

In [8]:

failed_pages=[]
with open("reviews.txt", "a") as file_handle:
    for pg in range(11):
        file_handle.write(f'\r\n # Page-{pg+1}\r\n')
        file_handle.flush()
        try:
            s=''
            # set the param of the following funtion to the review page number to extract
            print(f'------- SCRAPING PAGE {pg+1} -------')
            s = scrape_page((pg+1))
            if s == 'Retrieval error.':
                raise Exception(f'{s} Page index {pg}')
            file_handle.write(s)
            file_handle.flush()
        except Exception as e:
            print(f'ERROR OCCURED WHILE RETRIEVING PAGE {pg+1}:  {e}')
            failed_pages.append(pg)

file_handle.close()


------- SCRAPING PAGE 1 -------
Getting ... https://www.dell.com/en-in/work/shop/business-laptop-notebook-computers/latitude-7440-laptop/spd/latitude-14-7440-2-in-1-laptop/n001l7440017in9_vp?bvstate=pg:1/ct:r#ratings_section
Checking Shadow DOM ...
Trying 1 of 5 ...
Shadow DOM found.
Shadow DOM inner element ('#reviews_container') found.
Relevant information from page 1 retrieved.
------- SCRAPING PAGE 2 -------
Getting ... https://www.dell.com/en-in/work/shop/business-laptop-notebook-computers/latitude-7440-laptop/spd/latitude-14-7440-2-in-1-laptop/n001l7440017in9_vp?bvstate=pg:2/ct:r#ratings_section
Checking Shadow DOM ...
Trying 1 of 5 ...
Checking Shadow DOM ...
Trying 2 of 5 ...
Shadow DOM found.
Shadow DOM inner element ('#reviews_container') found.
Relevant information from page 2 retrieved.
------- SCRAPING PAGE 3 -------
Getting ... https://www.dell.com/en-in/work/shop/business-laptop-notebook-computers/latitude-7440-laptop/spd/latitude-14-7440-2-in-1-laptop/n001l7440017in9_vp

### <font color='a33668'> Printing the CSV for a given page </font>

- CSV Columns: Rating, Review Title, Review

In [9]:
failed_pages

[6, 9, 10]